In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
import json
from openai import OpenAI

In [3]:
sys.path.append(os.path.join(os.getcwd(), '..'))

In [4]:
from utils.select_enviroment import select_enviroment

In [59]:
select_enviroment()

Selected enviroment: d:\Workspace\ner_alternatives\src\lib\utils\..\.env


In [60]:
fine_tuning_data_dir = os.path.join(os.getcwd(), '..', '..', '..', 'resources')
rest_fine_tuning_data_path = os.path.join(fine_tuning_data_dir, 'rest_fine_tuning_data.jsonl')

In [61]:
client = OpenAI()

file = client.files.create(
  file=open(rest_fine_tuning_data_path, "rb"),
  purpose="fine-tune"
)

APIConnectionError: Connection error.

In [ ]:
client.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-4o-mini"
)

BadRequestError: Error code: 400 - {'error': {'message': 'Model gpt-4o-mini is not available for fine-tuning or does not exist.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}

# I am not able to fine-tune via api because i am not yet tier 4 or 5. 
# But OpenAI offers the possibility to fine-tune via their web with a beatiful UI.

In [3]:
def evaluate_model_output(expected_output, model_output):
    verif1_pass = True
    verif2_pass = True
    verif3_pass = True
    verif4_pass = True

    # Coleccionar todas las entidades esperadas y predichas independientemente de la categoría
    ref_texts_all = set(
        entity[0]
        for category in expected_output
        for entity in expected_output[category]
    )
    model_texts_all = set(
        entity[0] for category in model_output for entity in model_output[category]
    )
    print(ref_texts_all)
    print(model_texts_all)
    # Verificación 4: Identifica todas las entidades esperadas sin errores
    if ref_texts_all != model_texts_all:
        verif4_pass = False

    if verif4_pass:  # Solo evaluar si verif4_pass no ha fallado aún
        for category in expected_output:
            ref_entities = expected_output[category]
            model_entities = [tuple(entity) for entity in model_output.get(category, [])]

            # Verificación 3: Clasifica correctamente dichas entidades
            if verif3_pass:  # Solo evaluar si verif3_pass no ha fallado aún
                ref_set = {(entity[0], category) for entity in ref_entities}
                model_set = {(entity[0], category) for entity in model_entities}
                print(ref_set)
                print(model_set)
                if ref_set != model_set:
                    verif3_pass = False
            else:
                verif2_pass = False
                verif1_pass = False

            # Verificación 2: Las ubica bien dentro del texto con un margen de error de +-2
            if verif2_pass:  # Solo evaluar si verif2_pass no ha fallado aún
                for ref_entity in ref_entities:
                    ref_text, ref_start, ref_end = ref_entity
                    matched = False
                    for model_entity in model_entities:
                        model_text, model_start, model_end = model_entity
                        if (
                            ref_text == model_text
                            and abs(ref_start - model_start) <= 2
                            and abs(ref_end - model_end) <= 2
                        ):
                            matched = True
                            break
                    if not matched:
                        verif2_pass = False
                        break
            else:
                verif1_pass = False

            # Verificación 1: Las ubica bien dentro del texto
            if verif1_pass:  # Solo evaluar si verif1_pass no ha fallado aún
                print(ref_entities)
                print(model_entities)
                ref_set = set(ref_entities)
                model_set = set(model_entities)
                if ref_set != model_set:
                    verif1_pass = False
    else:
        verif3_pass = False
        verif2_pass = False
        verif1_pass = False

    return verif1_pass, verif2_pass, verif3_pass, verif4_pass


In [23]:
def evaluate_model_output_2(expected_output, model_output):
    verif1_pass = True
    verif2_pass = True
    verif3_pass = True
    verif4_pass = True

    # Coleccionar todas las entidades esperadas y predichas independientemente de la categoría
    ref_texts_all = set(
        entity[0]
        for category in expected_output
        for entity in expected_output[category]
    )
    model_texts_all = set(
        entity['name'] for category in model_output for entity in model_output[category]
    )
    
    # Verificación 4: Identifica todas las entidades esperadas sin errores
    if ref_texts_all != model_texts_all:
        verif4_pass = False

    if verif4_pass:  # Solo evaluar si verif4_pass no ha fallado aún
        for category in expected_output:
            ref_entities = expected_output[category]
            model_entities = [
                (entity['name'], entity['start_position'], entity['end_position']) 
                for entity in model_output.get(category, [])
            ]

            # Verificación 3: Clasifica correctamente dichas entidades
            if verif3_pass:  # Solo evaluar si verif3_pass no ha fallado aún
                ref_set = {(entity[0], category) for entity in ref_entities}
                model_set = {(entity[0], category) for entity in model_entities}
                if ref_set != model_set:
                    verif3_pass = False
            else:
                verif2_pass = False
                verif1_pass = False

            # Verificación 2: Las ubica bien dentro del texto con un margen de error de +-2
            if verif2_pass:  # Solo evaluar si verif2_pass no ha fallado aún
                for ref_entity in ref_entities:
                    ref_text, ref_start, ref_end = ref_entity
                    matched = False
                    for model_entity in model_entities:
                        model_text, model_start, model_end = model_entity
                        if (
                            ref_text == model_text
                            and abs(ref_start - model_start) <= 2
                            and abs(ref_end - model_end) <= 2
                        ):
                            matched = True
                            break
                    if not matched:
                        verif2_pass = False
                        break
            else:
                verif1_pass = False

            # Verificación 1: Las ubica bien dentro del texto
            if verif1_pass:  # Solo evaluar si verif1_pass no ha fallado aún
                ref_set = set(ref_entities)
                model_set = set(model_entities)
                if ref_set != model_set:
                    verif1_pass = False
    else:
        verif3_pass = False
        verif2_pass = False
        verif1_pass = False

    return verif1_pass, verif2_pass, verif3_pass, verif4_pass

In [24]:
#expected_output = {'adverse_drug_reactions': [('Extremely dry mouth', 0, 19), ('severe joint aches', 21, 39), ('muscle cramps in lower legs', 41, 68), ('difficulty breathing', 112, 132), ('tired', 134, 139), ('memory loss', 141, 152), ('mild depression', 154, 169), ('diminished eyesight', 171, 190), ('forgetfulness', 360, 373), ('memory failure', 396, 410), ('swelling', 525, 533), ('muscle aches', 559, 571), ('joint pain', 576, 586), ('memory loss', 600, 611)], 'diseases_or_medical_conditions': [], 'medications': [('Lipitor', 460, 467)], 'clinical_findings': [], 'symptoms_experienced_by_patients': []}
#output = {'adverse_drug_reactions': [['Extremely dry mouth', 0, 19], ['muscle cramps in lower legs', 44, 71], ['severe joint aches', 21, 39], ['sever swelling of feet, hands and ankles', 73, 114], ['severe, breathing is easier, stamina is back', 562, 605], ['difficulty breathing', 116, 136], ['memory loss', 148, 159], ['mild depression', 161, 176], ['diminished eyesight', 178, 198], ['swelling decreased significantly', 511, 542], ['muscle aches', 544, 556], ['joint pain disappeared', 561, 583], ['memory loss is not as', 585, 608]], 'diseases_or_medical_conditions': [], 'medications': [['Lipitor', 401, 408]], 'clinical_findings': [], 'symptoms_experienced_by_patients': [['forgetfulness', 316, 329], ['memory failure', 350, 364]]}
#output = {'adverse_drug_reactions': [{'name': 'extremely dry mouth', 'start_position': 0, 'end_position': 19}, {'name': 'severe joint aches', 'start_position': 21, 'end_position': 38}, {'name': 'muscle cramps in lower legs', 'start_position': 40, 'end_position': 67}, {'name': 'severe swelling of feet, hands and ankles', 'start_position': 69, 'end_position': 110}, {'name': 'difficulty breathing', 'start_position': 112, 'end_position': 131}, {'name': 'tired', 'start_position': 133, 'end_position': 138}, {'name': 'memory loss', 'start_position': 140, 'end_position': 151}, {'name': 'mild depression', 'start_position': 153, 'end_position': 168}, {'name': 'diminished eyesight', 'start_position': 170, 'end_position': 188}], 'diseases_or_medical_conditions': [], 'medications': [{'name': 'Lipitor', 'start_position': 378, 'end_position': 385}], 'clinical_findings': [], 'symptoms_experienced_by_patients': [{'name': 'extremely dry mouth', 'start_position': 0, 'end_position': 19}, {'name': 'severe joint aches', 'start_position': 21, 'end_position': 38}, {'name': 'muscle cramps in lower legs', 'start_position': 40, 'end_position': 67}, {'name': 'severe swelling of feet, hands and ankles', 'start_position': 69, 'end_position': 110}, {'name': 'difficulty breathing', 'start_position': 112, 'end_position': 131}, {'name': 'tired', 'start_position': 133, 'end_position': 138}, {'name': 'memory loss', 'start_position': 140, 'end_position': 151}, {'name': 'mild depression', 'start_position': 153, 'end_position': 168}, {'name': 'diminished eyesight', 'start_position': 170, 'end_position': 188}]}
#expected_output = {'adverse_drug_reactions': [('Extremely dry mouth', 0, 19), ('severe joint aches', 21, 39), ('muscle cramps in lower legs', 41, 68), ('difficulty breathing', 112, 132), ('tired', 134, 139), ('memory loss', 141, 152), ('mild depression', 154, 169), ('diminished eyesight', 171, 190), ('forgetfulness', 360, 373), ('memory failure', 396, 410), ('swelling', 525, 533), ('muscle aches', 559, 571), ('joint pain', 576, 586), ('memory loss', 600, 611)], 'diseases_or_medical_conditions': [], 'medications': [('Lipitor', 460, 467)], 'clinical_findings': [], 'symptoms_experienced_by_patients': []}
expected_output = {'adverse_drug_reactions': [('Extremely dry mouth', 0, 19), ('severe joint aches', 21, 39), ('muscle cramps in lower legs', 41, 68), ('difficulty breathing', 112, 132), ('tired', 134, 139), ('memory loss', 141, 152), ('mild depression', 154, 169), ('diminished eyesight', 171, 190), ('forgetfulness', 360, 373), ('memory failure', 396, 410), ('swelling', 525, 533), ('muscle aches', 559, 571), ('joint pain', 576, 586), ('memory loss', 600, 611)], 'diseases_or_medical_conditions': [], 'medications': [('Lipitor', 460, 467)], 'clinical_findings': [], 'symptoms_experienced_by_patients': []}
output = {'adverse_drug_reactions': [{'name': 'Extremely dry mouth', 'start_position': 0, 'end_position': 20}, {'name': 'severe joint aches', 'start_position': 22, 'end_position': 41}, {'name': 'muscle cramps in lower legs', 'start_position': 43, 'end_position': 72}, {'name': 'sever swelling of feet, hands and ankles', 'start_position': 74, 'end_position': 116}, {'name': 'difficulty breathing', 'start_position': 118, 'end_position': 137}, {'name': 'tired', 'start_position': 139, 'end_position': 144}, {'name': 'memory loss', 'start_position': 146, 'end_position': 157}, {'name': 'mild depression', 'start_position': 159, 'end_position': 175}, {'name': 'diminished eyesight', 'start_position': 177, 'end_position': 196}], 'diseases_or_medical_conditions': [{'name': 'mild depression', 'start_position': 159, 'end_position': 175}], 'medications': [{'name': 'Lipitor', 'start_position': 419, 'end_position': 426}], 'clinical_findings': [{'name': 'swelling decreased significantly', 'start_position': 494, 'end_position': 525}, {'name': 'muscle aches and joint pain disappeared', 'start_position': 527, 'end_position': 567}, {'name': 'memory loss is not as severe', 'start_position': 569, 'end_position': 599}, {'name': 'breathing is easier', 'start_position': 601, 'end_position': 620}, {'name': 'stamina is back', 'start_position': 622, 'end_position': 637}], 'symptoms_experienced_by_patients': [{'name': 'Extremely dry mouth', 'start_position': 0, 'end_position': 20}, {'name': 'severe joint aches', 'start_position': 22, 'end_position': 41}, {'name': 'muscle cramps in lower legs', 'start_position': 43, 'end_position': 72}, {'name': 'sever swelling of feet, hands and ankles', 'start_position': 74, 'end_position': 116}, {'name': 'difficulty breathing', 'start_position': 118, 'end_position': 137}, {'name': 'tired', 'start_position': 139, 'end_position': 144}, {'name': 'memory loss', 'start_position': 146, 'end_position': 157}, {'name': 'mild depression', 'start_position': 159, 'end_position': 175}, {'name': 'diminished eyesight', 'start_position': 177, 'end_position': 196}, {'name': 'forgetfulness', 'start_position': 329, 'end_position': 342}]}


In [25]:
verif1_model1, verif2_model1, verif3_model1, verif4_model1 = evaluate_model_output_2(expected_output, output)
print(f"Model 1 - Verificación 1: {verif1_model1}, Verificación 2: {verif2_model1}, Verificación 3: {verif3_model1}, Verificación 4: {verif4_model1}")

Model 1 - Verificación 1: False, Verificación 2: False, Verificación 3: False, Verificación 4: False


In [7]:
verif1_model1, verif2_model1, verif3_model1, verif4_model1 = evaluate_model_output(expected_output, output)
print(f"Model 1 - Verificación 1: {verif1_model1}, Verificación 2: {verif2_model1}, Verificación 3: {verif3_model1}, Verificación 4: {verif4_model1}")

KeyError: 0